In [23]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

from keras.applications.vgg16 import VGG16 # load VGG16 model from keras
from keras.preprocessing.image import load_img # Using images library
from keras.preprocessing.image import img_to_array # # convert image pixels to numpy for specific manipulations
from keras.applications.vgg16 import preprocess_input # to prepare for new input
from keras.applications.vgg16 import decode_predictions # for reporting probabilities

In [24]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# prepare an iterators for each dataset
train_it = train_datagen.flow_from_directory('data/images/train',
                                     target_size=(224,224),
                                     color_mode='rgb',
                                     batch_size=32,
                                     class_mode='categorical',
                                     shuffle=True)
# val_it = datagen.flow_from_directory('data/validation/', class_mode='categorical')
test_it = train_datagen.flow_from_directory('data/images/test',
                                     target_size=(224,224),
                                     color_mode='rgb',
                                     batch_size=32,
                                     class_mode='categorical',
                                     shuffle=True)
# confirm the iterator works
train_batchX, train_batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (train_batchX.shape, train_batchX.min(), train_batchX.max()))
test_batchX, test_batchy = test_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (test_batchX.shape, test_batchX.min(), test_batchX.max()))

Found 58 images belonging to 5 classes.
Found 16 images belonging to 5 classes.
Batch shape=(32, 224, 224, 3), min=-123.680, max=151.061
Batch shape=(16, 224, 224, 3), min=-123.680, max=151.061


In [25]:
# classifying only weapons 
vgg16_model = VGG16(weights="imagenet")

In [26]:
from keras.models import Sequential
from keras.layers import Dense, Activation

seq_model = Sequential()

# iterate all layers in vgg16 to sequential for custom modification
for layer in vgg16_model.layers:
    seq_model.add(layer)
    
seq_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [27]:
# 1000 categories, pop off output layer to work only specific categories
seq_model._layers.pop()

seq_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [28]:
# freeze weights to prevent constantly updating weights
for layer in seq_model._layers:
    layer.trainable = False

In [29]:
seq_model.add(Dense(5, activation='softmax')) # 5 categories 

seq_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [30]:
seq_model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train = train_it.n//train_it.batch_size
seq_model.fit_generator(generator=train_it,
                   steps_per_epoch=step_size_train,
                   epochs=10)

Epoch 1/10
1/1 [==============================] - 1s 541ms/step - loss: 1.6050 - accuracy: 0.0938
Epoch 2/10
1/1 [==============================] - 0s 180ms/step - loss: 1.6108 - accuracy: 0.1538
Epoch 3/10
1/1 [==============================] - 0s 291ms/step - loss: 1.5940 - accuracy: 0.5312
Epoch 4/10
1/1 [==============================] - 0s 179ms/step - loss: 1.5833 - accuracy: 0.6538
Epoch 5/10
1/1 [==============================] - 0s 344ms/step - loss: 1.5696 - accuracy: 0.8438
Epoch 6/10
1/1 [==============================] - 0s 180ms/step - loss: 1.5870 - accuracy: 0.4615
Epoch 7/10
1/1 [==============================] - 0s 204ms/step - loss: 1.5711 - accuracy: 0.6538
Epoch 8/10
1/1 [==============================] - 0s 234ms/step - loss: 1.5685 - accuracy: 0.7188
Epoch 9/10
1/1 [==============================] - 0s 243ms/step - loss: 1.5670 - accuracy: 0.7188
Epoch 10/10
1/1 [==============================] - 0s 177ms/step - loss: 1.5572 - accuracy: 0.7692


In [31]:
seq_model.predict(test_it)

array([[0.2026464 , 0.19908138, 0.1978932 , 0.2044389 , 0.19594012],
       [0.19768012, 0.20516607, 0.19276495, 0.21537927, 0.18900958],
       [0.20903952, 0.20583363, 0.19100256, 0.20641614, 0.18770821],
       [0.21570279, 0.2081697 , 0.18742748, 0.2030016 , 0.1856984 ],
       [0.19447419, 0.19905688, 0.1968556 , 0.20691983, 0.20269349],
       [0.19043697, 0.19901691, 0.21068533, 0.19415216, 0.20570856],
       [0.19502497, 0.18496108, 0.1994018 , 0.21005264, 0.21055952],
       [0.20072961, 0.20062801, 0.19491467, 0.20879872, 0.19492903],
       [0.20963065, 0.20343432, 0.1909075 , 0.20299709, 0.19303048],
       [0.2039522 , 0.19941919, 0.18882504, 0.2083729 , 0.19943075],
       [0.1931033 , 0.19980426, 0.19989271, 0.2021003 , 0.20509942],
       [0.20350012, 0.19819865, 0.19501932, 0.20819259, 0.1950893 ],
       [0.19035998, 0.1994283 , 0.20936893, 0.19547564, 0.20536712],
       [0.21383739, 0.1988841 , 0.19347116, 0.20083095, 0.19297644],
       [0.20696756, 0.20160645, 0.

In [32]:
seq_model.predict(test_batchX)   

array([[0.2026464 , 0.19908138, 0.1978932 , 0.2044389 , 0.19594012],
       [0.19768012, 0.20516607, 0.19276495, 0.21537927, 0.18900958],
       [0.20903952, 0.20583363, 0.19100256, 0.20641614, 0.18770821],
       [0.21570279, 0.2081697 , 0.18742748, 0.2030016 , 0.1856984 ],
       [0.19447419, 0.19905688, 0.1968556 , 0.20691983, 0.20269349],
       [0.19043697, 0.19901691, 0.21068533, 0.19415216, 0.20570856],
       [0.19502497, 0.18496108, 0.1994018 , 0.21005264, 0.21055952],
       [0.20072961, 0.20062801, 0.19491467, 0.20879872, 0.19492903],
       [0.20963065, 0.20343432, 0.1909075 , 0.20299709, 0.19303048],
       [0.2039522 , 0.19941919, 0.18882504, 0.2083729 , 0.19943075],
       [0.1931033 , 0.19980426, 0.19989271, 0.2021003 , 0.20509942],
       [0.20350012, 0.19819865, 0.19501932, 0.20819259, 0.1950893 ],
       [0.19035998, 0.1994283 , 0.20936893, 0.19547564, 0.20536712],
       [0.21383739, 0.1988841 , 0.19347116, 0.20083095, 0.19297644],
       [0.20696756, 0.20160645, 0.

In [33]:
prediction = seq_model.predict(test_batchX)

# convert the probabilities to class labels
#label = decode_predictions(pred)

for row in range(len(prediction)):
    for val in range(len(prediction[row])):
        if(val == np.argmax(prediction[row])):
            prediction[row][val] = 1.
        else:
            prediction[row][val] = 0.

# retrieve the most likely result, e.g. highest probability
prediction == test_batchy

array([[False,  True,  True, False,  True],
       [ True,  True,  True,  True,  True],
       [False,  True,  True,  True, False],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True, False, False],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True, False,  True, False,  True],
       [False,  True,  True,  True, False],
       [ True,  True, False, False,  True],
       [ True,  True, False,  True, False],
       [ True,  True,  True, False, False],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True, False, False]])